In [1]:
is_notebook = get_ipython().has_trait("kernel")  # type: ignore

In [2]:
disk_map = input() or "2333133121414131402"

The disk map alternates between files and spaces, starting with files.


In [3]:
from dataclasses import dataclass
from typing import Union


@dataclass
class File:
    id: int
    size: int


@dataclass
class Space:
    size: int


DiskElement = Union[File, Space]

In [4]:
from typing import Iterable

In [5]:
def expand_disk_map(disk_map: str) -> Iterable[DiskElement]:
    for i, element_count in enumerate(disk_map):
        is_file = i % 2 == 0
        element_count = int(element_count)

        if is_file:
            yield File(id=i // 2, size=element_count)
        else:
            yield Space(size=element_count)

In [6]:
def format_disk(disk: Iterable[DiskElement]) -> str:
    return "".join(
        f"{element.id}" * element.size
        if isinstance(element, File)
        else "." * element.size
        for element in disk
    )

In [7]:
format_disk(expand_disk_map("12345"))

'0..111....22222'

In [8]:
format_disk(expand_disk_map(disk_map)) if is_notebook else None

'00...111...2...333.44.5555.6666.777.888899'

In [9]:
def find_space(disk: list[DiskElement], size: int) -> int:
    for i, element in enumerate(disk):
        if isinstance(element, Space) and element.size >= size:
            return i
    return -1

In [12]:
find_space(list(expand_disk_map("12345")), 3)

3

In [21]:
def move_file(disk: list[DiskElement], from_index: int, to_index: int) -> None:
    assert from_index > to_index

    file = disk[from_index]
    disk[from_index] = Space(size=file.size)

    disk[to_index].size -= file.size
    if disk[to_index].size == 0:
        disk[to_index] = file
    else:
        disk.insert(to_index, file)

In [25]:
if is_notebook:
    test_disk = list(expand_disk_map(disk_map))
    print(format_disk(test_disk))
    nine_index = next(
        i
        for i, element in enumerate(test_disk)
        if isinstance(element, File) and element.id == 9
    )
    move_file(test_disk, from_index=nine_index, to_index=1)
    print(format_disk(test_disk))

00...111...2...333.44.5555.6666.777.888899
0099.111...2...333.44.5555.6666.777.8888..


In [37]:
def defrag_disk(
    disk: Iterable[DiskElement],
) -> Iterable[list[DiskElement]]:
    result = list(disk)
    file_ids = [x.id for x in result if isinstance(x, File)]

    yield result

    while file_ids:
        file_id = file_ids.pop()
        file_index = next(
            i for i, x in enumerate(result) if isinstance(x, File) and x.id == file_id
        )
        file = result[file_index]
        space_index = find_space(result, file.size)

        if space_index == -1 or space_index >= file_index:
            continue

        move_file(result, from_index=file_index, to_index=space_index)

        yield result

In [41]:
it = defrag_disk(expand_disk_map(disk_map))

[format_disk(step) for step in it] if is_notebook else None

['00...111...2...333.44.5555.6666.777.888899',
 '0099.111...2...333.44.5555.6666.777.8888..',
 '0099.1117772...333.44.5555.6666.....8888..',
 '0099.111777244.333....5555.6666.....8888..',
 '00992111777.44.333....5555.6666.....8888..']

In [42]:
*_, defragged = defrag_disk(list(expand_disk_map(disk_map)))

In [46]:
def checksum(disk: Iterable[DiskElement]) -> Iterable[int]:
    block = 0
    for x in disk:
        for _ in range(x.size):
            if isinstance(x, File):
                yield block * x.id
            block += 1

In [47]:
print(sum(checksum(defragged)))

2858
